# Heat equation example

## Analytic problem formulation

We consider the heat equation on the segment $[0, 1]$, with dissipation on both sides, heating (input) $u$ on the left, and measurement (output) $\tilde{y}$ on the right:
$$
\begin{align*}
    \partial_t T(z, t) & = \partial_{zz} T(z, t), & 0 < z < 1,\ t > 0, \\
    \partial_z T(0, t) & = T(0, t) - u(t), & t > 0, \\
    \partial_z T(1, t) & = -T(1, t), & t > 0, \\
    \tilde{y}(t) & = T(1, t), & t > 0.
\end{align*}
$$


## Import modules

In [ ]:
import numpy as np
import scipy.linalg as spla
import scipy.integrate as spint
import matplotlib.pyplot as plt

from pymor.basic import *
from pymor.core.config import config
set_log_levels({'pymor.algorithms.gram_schmidt.gram_schmidt': 'WARNING'})

## Assemble LTISystem

### Discretize problem

In [ ]:
p = InstationaryProblem(
    StationaryProblem(
        domain=LineDomain([0.,1.], left='robin', right='robin'),
        diffusion=ConstantFunction(1., 1),
        robin_data=(ConstantFunction(1., 1), ExpressionFunction('(x[...,0] < 1e-10) * 1.', 1)),
        functionals={'output': ('l2_boundary', ExpressionFunction('(x[...,0] > (1 - 1e-10)) * 1.', 1))}
    ),
    ConstantFunction(0., 1),
    T=3.
)

d, _ = discretize_instationary_cg(p, diameter=1/100, nt=100)

### Visualize solution for constant input of 1

In [ ]:
d.visualize(d.solve())

### Convert to LTISystem

In [ ]:
lti = d.to_lti()

## LTI system

In [ ]:
print('n = {}'.format(lti.n))
print('m = {}'.format(lti.m))
print('p = {}'.format(lti.p))

In [ ]:
poles = lti.poles()
fig, ax = plt.subplots()
ax.plot(poles.real, poles.imag, '.')
ax.set_title('System poles')
plt.show()

In [ ]:
w = np.logspace(-2, 3, 100)
fig, ax = plt.subplots()
lti.mag_plot(w, ax=ax)
ax.set_title('Bode plot of the full model')
plt.show()

In [ ]:
hsv = lti.hsv()
fig, ax = plt.subplots()
ax.semilogy(range(1, len(hsv) + 1), hsv, '.-')
ax.set_title('Hankel singular values')
plt.show()

In [ ]:
print('H_2-norm of the full model:    {:e}'.format(lti.h2_norm()))
if config.HAVE_SLYCOT:
    print('H_inf-norm of the full model:  {:e}'.format(lti.hinf_norm()))
print('Hankel-norm of the full model: {:e}'.format(lti.hankel_norm()))

## Balanced Truncation (BT)

In [ ]:
r = 5
bt_reductor = BTReductor(lti)
rom_bt = bt_reductor.reduce(r, tol=1e-5)

In [ ]:
err_bt = lti - rom_bt
print('H_2-error for the BT ROM:    {:e}'.format(err_bt.h2_norm()))
if config.HAVE_SLYCOT:
    print('H_inf-error for the BT ROM:  {:e}'.format(err_bt.hinf_norm()))
print('Hankel-error for the BT ROM: {:e}'.format(err_bt.hankel_norm()))

In [ ]:
fig, ax = plt.subplots()
lti.mag_plot(w, ax=ax)
rom_bt.mag_plot(w, ax=ax, linestyle='dashed')
ax.set_title('Bode plot of the full and BT reduced model')
plt.show()

In [ ]:
fig, ax = plt.subplots()
err_bt.mag_plot(w, ax=ax)
ax.set_title('Bode plot of the BT error system')
plt.show()

## LQG Balanced Truncation (LQGBT)

In [ ]:
r = 5
lqgbt_reductor = LQGBTReductor(lti)
rom_lqgbt = lqgbt_reductor.reduce(r, tol=1e-5)

In [ ]:
err_lqgbt = lti - rom_lqgbt
print('H_2-error for the LQGBT ROM:    {:e}'.format(err_lqgbt.h2_norm()))
if config.HAVE_SLYCOT:
    print('H_inf-error for the LQGBT ROM:  {:e}'.format(err_lqgbt.hinf_norm()))
print('Hankel-error for the LQGBT ROM: {:e}'.format(err_lqgbt.hankel_norm()))

In [ ]:
fig, ax = plt.subplots()
lti.mag_plot(w, ax=ax)
rom_lqgbt.mag_plot(w, ax=ax, linestyle='dashed')
ax.set_title('Bode plot of the full and LQGBT reduced model')
plt.show()

In [ ]:
fig, ax = plt.subplots()
err_lqgbt.mag_plot(w, ax=ax)
ax.set_title('Bode plot of the LGQBT error system')
plt.show()

## Bounded Real Balanced Truncation (BRBT)

In [ ]:
r = 5
brbt_reductor = BRBTReductor(lti, 0.34)
rom_brbt = brbt_reductor.reduce(r, tol=1e-5)

In [ ]:
err_brbt = lti - rom_brbt
print('H_2-error for the BRBT ROM:    {:e}'.format(err_brbt.h2_norm()))
if config.HAVE_SLYCOT:
    print('H_inf-error for the BRBT ROM:  {:e}'.format(err_brbt.hinf_norm()))
print('Hankel-error for the BRBT ROM: {:e}'.format(err_brbt.hankel_norm()))

In [ ]:
fig, ax = plt.subplots()
lti.mag_plot(w, ax=ax)
rom_brbt.mag_plot(w, ax=ax, linestyle='dashed')
ax.set_title('Bode plot of the full and BRBT reduced model')
plt.show()

In [ ]:
fig, ax = plt.subplots()
err_brbt.mag_plot(w, ax=ax)
ax.set_title('Bode plot of the BRBT error system')
plt.show()

## Iterative Rational Krylov Algorithm (IRKA)

In [ ]:
r = 5
sigma = np.logspace(-1, 3, r)
tol = 1e-4
maxit = 100

irka_reductor = IRKAReductor(lti)
rom_irka = irka_reductor.reduce(r, sigma, tol=tol, maxit=maxit, compute_errors=True)

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(irka_reductor.dist, '.-')
ax.set_title('Distances between shifts in IRKA iterations')
plt.show()

In [ ]:
err_irka = lti - rom_irka
print('H_2-error for the IRKA ROM:    {:e}'.format(err_irka.h2_norm()))
if config.HAVE_SLYCOT:
    print('H_inf-error for the IRKA ROM:  {:e}'.format(err_irka.hinf_norm()))
print('Hankel-error for the IRKA ROM: {:e}'.format(err_irka.hankel_norm()))

In [ ]:
fig, ax = plt.subplots()
lti.mag_plot(w, ax=ax)
rom_irka.mag_plot(w, ax=ax, linestyle='dashed')
ax.set_title('Bode plot of the full and IRKA reduced model')
plt.show()

In [ ]:
fig, ax = plt.subplots()
err_irka.mag_plot(w, ax=ax)
ax.set_title('Bode plot of the IRKA error system')
plt.show()

## Two-Sided Iteration Algorithm (TSIA)

In [ ]:
r = 5
Ar = np.diag(-np.logspace(-1, 3, r))
Br = np.ones((r, 1))
Cr = np.ones((1, r))
Er = np.eye(r)

rom0 = LTISystem.from_matrices(Ar, Br, Cr, E=Er,
                               input_id=lti.input_space.id,
                               state_id=lti.state_space.id,
                               output_id=lti.output_space.id)

tsia_reductor = TSIAReductor(lti)
rom_tsia = tsia_reductor.reduce(rom0, compute_errors=True)

## Transfer Function IRKA (TF-IRKA)

Applying Laplace transformation to the original PDE formulation, we obtain a parametric boundary value problem
$$
\begin{align*}
    s \hat{T}(z, s) & = \partial_{zz} \hat{T}(z, s), \\
    \partial_z \hat{T}(0, s) & = \hat{T}(0, s) - \hat{u}(s), \\
    \partial_z \hat{T}(1, s) & = -\hat{T}(1, s), \\
    \hat{\tilde{y}}(s) & = \hat{T}(1, s),
\end{align*}
$$
where $\hat{T}$, $\hat{u}$, and $\hat{\tilde{y}}$ are respectively Laplace transforms of $T$, $u$, and $\tilde{y}$.
We assumed the initial condition to be zero ($T(z, 0) = 0$).
The parameter $s$ is any complex number in the region convergence of the Laplace tranformation.

Inserting $\hat{T}(z, s) = c_1 \exp\left(\sqrt{s} z\right) + c_2 \exp\left(-\sqrt{s} z\right)$, from the boundary conditions we get a system of equations
$$
\begin{align*}
    \left(\sqrt{s} - 1\right) c_1
    - \left(\sqrt{s} + 1\right) c_2 + \hat{u}(s) & = 0, \\
    \left(\sqrt{s} + 1\right) \exp\left(\sqrt{s}\right) c_1
    - \left(\sqrt{s} - 1\right) \exp\left(-\sqrt{s}\right) c_2 & = 0.
\end{align*}
$$
We can solve it using `sympy` and then find the transfer function ($\hat{\tilde{y}}(s) / \hat{u}(s)$).

In [ ]:
import sympy as sy
sy.init_printing()

sy_s, sy_u, sy_c1, sy_c2 = sy.symbols('s u c1 c2')

sol = sy.solve([(sy.sqrt(sy_s) - 1) * sy_c1 - (sy.sqrt(sy_s) + 1) * sy_c2 + sy_u,
                (sy.sqrt(sy_s) + 1) * sy.exp(sy.sqrt(sy_s)) * sy_c1 -
                (sy.sqrt(sy_s) - 1) * sy.exp(-sy.sqrt(sy_s)) * sy_c2],
               [sy_c1, sy_c2])

y = sol[sy_c1] * sy.exp(sy.sqrt(sy_s)) + sol[sy_c2] * sy.exp(-sy.sqrt(sy_s))

tf = sy.simplify(y / sy_u)
tf

Notice that for $s = 0$, the expression is of the form $0 / 0$.

In [ ]:
sy.limit(tf, sy_s, 0)

In [ ]:
dtf = tf.diff(sy_s)
dtf

In [ ]:
sy.limit(dtf, sy_s, 0)

We can now form the transfer function system.

In [ ]:
def H(s):
    if s == 0:
        return np.array([[1 / 3]])
    else:
        return np.array([[complex(tf.subs(sy_s, s))]])

In [ ]:
def dH(s):
    if s == 0:
        return np.array([[-13 / 54]])
    else:
        return np.array([[complex(dtf.subs(sy_s, s))]])

In [ ]:
tf_sys = TransferFunction(NumpyVectorSpace(1, 'INPUT'), NumpyVectorSpace(1, 'OUTPUT'), H, dH)

Here we compare it to the discretized system, by Bode plot, $\mathcal{H}_2$-norm, and $\mathcal{H}_2$-distance.

In [ ]:
tf_sys_w = tf_sys.bode(w)
lti_w = lti.bode(w)

In [ ]:
fig, ax = plt.subplots()
ax.loglog(w, spla.norm(tf_sys_w - lti_w, axis=(1, 2)))
ax.set_title('Distance between PDE and discretized transfer function')
plt.show()

In [ ]:
tf_H2_int, int_err = spint.quad(lambda w: spla.norm(tf_sys.eval_tf(w * 1j)) ** 2, -np.inf, np.inf)
print((tf_H2_int, int_err))

In [ ]:
print('H_2-norm of the transfer function  = {:e}'.format(np.sqrt(tf_H2_int / 2 / np.pi)))
print('H_2-norm of the discretized system = {:e}'.format(lti.h2_norm()))

In [ ]:
dist_H2_int, dist_int_err = spint.quad(lambda w: spla.norm(tf_sys.eval_tf(w * 1j) - lti.eval_tf(w * 1j)) ** 2,
                                       -np.inf, np.inf, epsabs=1e-16)
print((dist_H2_int, dist_int_err))

In [ ]:
print('H_2-distance = {:e}'.format(np.sqrt(dist_H2_int / 2 / np.pi)))

TF-IRKA finds a reduced model from the transfer function.

In [ ]:
tf_irka_reductor = TF_IRKAReductor(tf_sys)
rom_tf_irka = tf_irka_reductor.reduce(r)

Here we compute the $\mathcal{H}_2$-distance from the original PDE model to the TF-IRKA's reduced model and to the IRKA's reduced model.

In [ ]:
error_H2, error_int_err = spint.quad(lambda w: spla.norm(tf_sys.eval_tf(w * 1j) -
                                                         rom_tf_irka.eval_tf(w * 1j)) ** 2,
                                     -np.inf, np.inf, epsabs=1e-16)
print((error_H2, error_int_err))

In [ ]:
print('H_2-error of TF-IRKA ROM = {:e}'.format(np.sqrt(error_H2 / 2 / np.pi)))

In [ ]:
error_irka_H2, error_irka_int_err = spint.quad(lambda w: spla.norm(tf_sys.eval_tf(w * 1j) -
                                                                   rom_irka.eval_tf(w * 1j)) ** 2,
                                               -np.inf, np.inf, epsabs=1e-16)
print((error_irka_H2, error_irka_int_err))

In [ ]:
print('H_2-error of IRKA ROM = {:e}'.format(np.sqrt(error_irka_H2 / 2 / np.pi)))